In [3]:
import pandas as pd

- [Click-through ad data from Kaggle competition](https://www.kaggle.com/c/avazu-ctr-prediction/data)
- train_subset is first 10K rows of 6+GB set

In [4]:
df = pd.read_csv('data/train_subset.csv')

In [31]:
df.head(3)

id  click      hour    C1  banner_pos   site_id site_domain  \
0  1.000009e+18      0  14102100  1005           0  1fbe01fe    f3845767   
1  1.000017e+19      0  14102100  1005           0  1fbe01fe    f3845767   
2  1.000037e+19      0  14102100  1005           0  1fbe01fe    f3845767   

  site_category    app_id app_domain ...  device_type device_conn_type    C14  \
0      28905ebd  ecad2386   7801e8d9 ...            1                2  15706   
1      28905ebd  ecad2386   7801e8d9 ...            1                0  15704   
2      28905ebd  ecad2386   7801e8d9 ...            1                0  15704   

   C15  C16   C17  C18  C19     C20  C21  
0  320   50  1722    0   35      -1   79  
1  320   50  1722    0   35  100084   79  
2  320   50  1722    0   35  100084   79  

[3 rows x 24 columns]

In [6]:
# how many features should we have after?
len(df['device_id'].unique())

7201

Features are $\theta$ = [$N^+$, $N^-$, $log(N^+)-log(N^-)$, isRest]

$N^+$ = $p(+)$ = $n^+/(n^+ + n^-)$

$N^-$ = $p(-)$ = $n^-/(n^+ + n^-)$

$log(N^+)-log(N^-)$ = $\frac{p(+)}{p(-)}$

isRest = back-off bin (not shown here)

In [47]:
def click_counting(x, bin_column):
    clicks = pd.Series(x[x['click'] > 0][bin_column].value_counts(), name='clicks')
    no_clicks = pd.Series(x[x['click'] < 1][bin_column].value_counts(), name='no_clicks')
    
    counts = pd.DataFrame([clicks,no_clicks]).T.fillna('0')
    counts['total'] = counts['clicks'].astype('int64') + counts['no_clicks'].astype('int64')
    
    return counts

def bin_counting(counts):
    counts['N+'] = counts['clicks'].astype('int64').divide(counts['total'].astype('int64'))
    counts['N-'] = counts['no_clicks'].astype('int64').divide(counts['total'].astype('int64'))
    counts['log_N+'] = counts['N+'].divide(counts['N-'])

#    If we wanted to only return bin-counting properties, we would filter here
    bin_counts = counts.filter(items= ['N+', 'N-', 'log_N+'])
    return counts, bin_counts

In [48]:
# bin counts example: device_id
bin_column = 'device_id'
device_clicks = click_counting(df.filter(items= [bin_column, 'click']), bin_column)
device_all, device_bin_counts = bin_counting(device_clicks)

In [43]:
# check to make sure we have all the devices
len(device_bin_counts)

7201

In [50]:
device_all.sort_values(by = 'total', ascending=False).head(4)

clicks no_clicks  total        N+        N-    log_N+
a99f214a  15729     71206  86935  0.180928  0.819072  0.220894
c357dbff     33       134    167  0.197605  0.802395  0.246269
31da1bd0      0        62     62  0.000000  1.000000  0.000000
936e92fb      5        54     59  0.084746  0.915254  0.092593

In [36]:
# We can see how this can change model evaluation time by comparing raw vs. bin-counting size
from sys import getsizeof

print('Our pandas Series, in bytes: ', getsizeof(df.filter(items= ['device_id', 'click'])))
print('Our bin-counting feature, in bytes: ', getsizeof(device_bin_counts))

Our pandas Series, in bytes:  7300031
Our bin-counting feature, in bytes:  525697
